In [ ]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
_df_lab = pd.DataFrame()
TRACK_END = 9.6
SUBFRAMES = 18
SLOW_FACTOR = 20
TRIALS = 10

ACCEL_MIN = 0.1
ACCEL_MAX = 3.9


In [ ]:
def build_frames(mass, fric):
    frames = []
    xs, ys = [], []

    accel_trials = np.linspace(ACCEL_MIN, ACCEL_MAX, TRIALS)

    ideal_forces = mass * accel_trials + fric
    noise_scale = 0.05 * ideal_forces + 0.05
    forces = ideal_forces + np.random.normal(0, noise_scale)

    frame_id = 0

    for i, (a, f) in enumerate(zip(accel_trials, forces)):
        T = np.sqrt(2 * TRACK_END / a)
        ts = np.linspace(0, T * SLOW_FACTOR, SUBFRAMES)

        for t in ts:
            x = 0.5 * a * (t / SLOW_FACTOR) ** 2

            frames.append(
                go.Frame(
                    data=[
                        go.Scatter(x=[0, 10], y=[0, 0]),
                        go.Scatter(x=[x], y=[0]),
                        go.Scatter(x=xs, y=ys, xaxis="x2", yaxis="y2"),
                    ],
                    layout=go.Layout(
                        annotations=[
                            dict(
                                x=5, y=0.85,
                                xref="x", yref="y",
                                showarrow=False,
                                font=dict(size=14),
                                text=(
                                    f"<b>Trial {i+1}/{TRIALS}</b><br>"
                                    f"Acceleration: {a:.2f} m/s²<br>"
                                    f"Measured Force: {f:.2f} N<br>"
                                    f"Mass: {mass:.2f} kg<br>"
                                    f"Friction: {fric:.2f} N"
                                )
                            )
                        ]
                    ),
                    name=str(frame_id),
                )
            )
            frame_id += 1

        xs.append(a)
        ys.append(f)

    return frames


In [ ]:
fig = go.Figure(
    data=[
        go.Scatter(x=[0, 10], y=[0, 0], mode="lines", line=dict(width=8)),
        go.Scatter(x=[0], y=[0], mode="markers", marker=dict(size=22)),
        go.Scatter(x=[], y=[], mode="markers", xaxis="x2", yaxis="y2"),
    ],
    layout=go.Layout(
        width=1100,
        height=620,
        showlegend=False,
        grid=dict(rows=1, columns=2, pattern="independent"),
        xaxis=dict(range=[0, 10], title="Position (m)"),
        yaxis=dict(visible=False),
        xaxis2=dict(range=[0, 4], title="Acceleration (m/s²)", fixedrange=True),
        yaxis2=dict(range=[0, 22], title="Measured Force (N)", fixedrange=True),
        updatemenus=[
            dict(
                type="buttons",
                x=0.5,
                y=-0.22,
                xanchor="center",
                buttons=[
                    dict(
                        label="▶ Play",
                        method="animate",
                        args=[
                            None,
                            {
                                "frame": {"duration": 40, "redraw": True},
                                "transition": {"duration": 0},
                                "fromcurrent": False,
                            },
                        ],
                    )
                ],
            )
        ],
    ),
)


In [ ]:
mass_slider = widgets.FloatSlider(
    value=1.2, min=0.3, max=4.0, step=0.3,
    description="Mass (kg)", continuous_update=False
)

fric_slider = widgets.FloatSlider(
    value=1.5, min=0.0, max=5.0, step=0.5,
    description="Friction (N)", continuous_update=False
)

build_button = widgets.Button(
    description="🔧 Build Experiment",
    button_style="info"
)


In [ ]:
def build_experiment(_):
    global _df_lab

    mass = mass_slider.value
    fric = fric_slider.value

    accel_trials = np.linspace(ACCEL_MIN, ACCEL_MAX, TRIALS)

    ideal_forces = mass * accel_trials + fric
    noise_scale = 0.05 * ideal_forces + 0.05
    forces = ideal_forces + np.random.normal(0, noise_scale)

    # ✅ SAVE DATA IN ENGINE STATE
    _df_lab = pd.DataFrame({
        "acceleration": accel_trials,
        "force": forces,
        "mass": mass,
        "friction": fric,
    })

    # ✅ BUILD ANIMATION
    fig.frames = build_frames(mass, fric)

    fig.layout.title = (
        "<b>Newton’s Second Law Virtual Lab</b><br>"
        f"Mass = {mass:.2f} kg, Friction = {fric:.2f} N"
    )

    print("✅ Experiment built. Rows:", len(_df_lab))
def get_df_lab():
    return _df_lab.copy()


In [ ]:
def show_sliders():
    display(mass_slider, fric_slider, build_button)

def show_plot():
    if fig.frames is None or len(fig.frames) == 0:
        print("⚠️ Please click Build Experiment first.")
        return
    display(fig)

